# Bootes Brown photo-z cross

The photometric redshifts are calculated for the Brown aperture matched catalogue. Since in the manufacture of the masterlist we take the Brown aperture matched catalogue and add in some other obejcts. We here asign the Brown objects with their help id.

NOTE: There is some confusion as to whether the naming of the ID columns which may have caused misassociations. This is currently under investigation and the photo-z may be superseeded.

## 1. Load catalogues and join on IRAC ID

There were various issues with the IDs. It now seems that 'ID' is some function of 'ID_SPITZER' and 'ID_OPTICAL'. All three of which contain duplicates. We join in on these coloums which I believe have the following relation to those used in dmu1

| name in dmu24 file      | name in dmu1 file    | notes    |
| :------------- | :----------: | -----------: |
|  'ID' |     |  Not present in dmu1 made here from multiple columns    |
| 'ID_SPITZER'   | 'brown_irac_id'|  |
| 'ID_OPTICAL'   | 'brown_i_id'|  |

We join on the 'brown_i_id' if present and join the remainder on 'brown_irac_id'.

In [1]:
from astropy.table import Table, join, Column, vstack
import numpy as np

In [2]:
photo_z = Table.read('./data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public.fits')[
    'ID',
    'ID_SPITZER',
    'ID_OPTICAL', 
    'z1_median'
]
cross_ids = Table.read('../../dmu1/dmu1_ml_Bootes/data/'
                     + 'master_list_cross_ident_bootes_20190701.fits')[
    'help_id', 
    'brown_i_id',
    'brown_irac_id'
]
#cross_ids['brown_i_id'].name = 'ID'


photo_z['ID_OPTICAL'].fill_value = -99
photo_z['ID_SPITZER'].fill_value = -99
photo_z = photo_z.filled()
photo_z['brown_i_id'] = photo_z['ID_OPTICAL']  #?????
photo_z['brown_irac_id'] = photo_z['ID_SPITZER']       #????
#photo_z = photo_z[photo_z['brown_i_id'] > -1]

In [3]:
print(
    len(photo_z['brown_i_id'][photo_z['brown_i_id']>-1]) , 
    len(np.unique(photo_z['brown_i_id'][photo_z['brown_i_id']>-1]))
)

2067406 2066890


In [4]:
print(
    len(photo_z['brown_irac_id'][photo_z['brown_irac_id']>-1]) , 
    len(np.unique(photo_z['brown_irac_id'][photo_z['brown_irac_id']>-1]))
)

147439 147439


In [5]:
cross_ids[:5]

help_id,brown_i_id,brown_irac_id
bytes27,int64,int64
HELP_J142500.285+331035.838,394263,-1
HELP_J143318.051+332158.118,473154,-1
HELP_J142939.235+354635.105,2019839,-1
HELP_J142446.142+331756.854,445407,-1
HELP_J142639.609+354724.470,2027735,-1


In [6]:
photo_z[:5]

ID,ID_SPITZER,ID_OPTICAL,z1_median,brown_i_id,brown_irac_id
int64,int64,int64,float64,int64,int64
0,-99,1,3.6887,1,-99
1,-99,2,0.2606,2,-99
2,-99,3,4.9513,3,-99
3,-99,4,4.4549,4,-99
4,-99,5,4.4835,5,-99


In [7]:
new_cat_i = join(
    photo_z['brown_i_id','z1_median'][photo_z['brown_i_id']>-1],
    cross_ids, 
    join_type='left', keys='brown_i_id'
)
new_cat_irac_only = join(
    photo_z['brown_irac_id','z1_median'][photo_z['brown_i_id']<0],
    cross_ids, 
    join_type='left', keys='brown_irac_id'
)
new_cat=vstack([new_cat_i,new_cat_irac_only])

#new_cat= new_cat_i

## 2. Replace bad photo-zs with nans

In [8]:
new_cat[new_cat['z1_median']<0]['z1_median'] = np.nan

In [9]:
new_cat['help_id'].fill_value = ''
new_cat = new_cat.filled()

## 3. Remove objects with duplicate help_ids

There seem to be a number of objects with duplicate ids which becaome duplicate help_ids. As far as I can tell these are totally identical so I remove the duplicates.

In [10]:
np.sum(new_cat['help_id'] == '')

0

In [11]:
new_cat = new_cat[~(new_cat['help_id'] == '')]

In [12]:
new_cat['help_id'].dtype

dtype('S27')

In [13]:
len(new_cat)

2214845

In [14]:
len(np.unique(new_cat['help_id']))

1657471

In [15]:
new_cat.sort('help_id')
len(new_cat['help_id'][:-1][(new_cat['help_id'][1:] == new_cat['help_id'][:-1])])

557374

In [16]:
new_cat[new_cat['help_id']=='HELP_J143841.165+335004.237']

brown_i_id,z1_median,help_id,brown_irac_id
int64,float64,bytes27,int64
778164,1.0382,HELP_J143841.165+335004.237,-1
778164,1.0382,HELP_J143841.165+335004.237,-1


In [17]:
is_duplicate = new_cat['help_id'][1:] == new_cat['help_id'][:-1]
#np.append(is_duplicate,False)
new_cat = new_cat[~np.append(is_duplicate,False)]

In [18]:
new_cat[new_cat['help_id']=='HELP_J143841.165+335004.237']

brown_i_id,z1_median,help_id,brown_irac_id
int64,float64,bytes27,int64
778164,1.0382,HELP_J143841.165+335004.237,-1


In [19]:
len(new_cat)

1657471

In [20]:
len(new_cat) == len(np.unique(new_cat['help_id']))

True

In [21]:
new_cat[:10].show_in_notebook()

idx,brown_i_id,z1_median,help_id,brown_irac_id
0,299759,0.7223,0.0,999999
1,1788580,0.9797,HELP_J142433.597+352308.103,-1
2,1891099,0.8861,HELP_J142433.602+353326.121,-1
3,1997888,0.7447,HELP_J142433.609+354416.174,-1
4,1776606,1.8403,HELP_J142433.611+352156.195,-1
5,1780409,0.3033,HELP_J142433.614+352219.215,-1
6,2003639,1.7235,HELP_J142433.614+354452.719,-1
7,1729999,0.3968,HELP_J142433.625+351736.189,-1
8,1888110,2.416,HELP_J142433.629+353308.009,-1
9,2000812,0.7871,HELP_J142433.635+354435.055,-1


## 4. Write catalogue
This will be the catalogues that is used in dmu32 to merge photozs

In [22]:
new_cat.write('./data/Bootes_opt_spitzer_merged_vac_opt3as_irac4as_all_public_with_help_id_20200923.fits', overwrite=True)